## Импорт библиотек и настройка путей

In [2]:
# Импорт библиотек
import pandas as pd
import numpy as np
from pathlib import Path
import os

# Настройки отображения
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 30)
pd.options.display.float_format = '{:.2f}'.format

# Инициализация путей
root = Path(os.getcwd()).parent.parent
data_raw = root / 'data' / 'raw_data'
data_out = root / 'data' / 'processed_data'

print("Пути инициализированы:")
print(f"Raw data: {data_raw}")
print(f"Processed data: {data_out}")

Пути инициализированы:
Raw data: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/raw_data
Processed data: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/processed_data


## Загрузка данных

In [5]:
# Загрузка данных
print("\nЗагружаем сырые данные...")
sessions_raw = pd.read_pickle(data_raw / 'ga_sessions.pkl')
hits_raw = pd.read_pickle(data_raw / 'ga_hits.pkl')
print("\nДанные успешно загружены!")


Загружаем сырые данные...

Данные успешно загружены!


## Первичный анализ данных

In [7]:
 def generate_summary(df: pd.DataFrame, name: str, sample_size: int = 3) -> None:
    """Генерация расширенной сводки по данным примерами"""
    print(f"\n{'='*50} {name.upper()} {'='*50}")
    print(f"Общее количество записей: {df.shape[0]:,}")
    print(f"Количество признаков: {df.shape[1]}")
    
    # Типы данных
    print("\nТипы данных:")
    print(df.dtypes.value_counts().rename('count').to_frame())
    
    # Пропуски
    missing = df.isna().sum().sort_values(ascending=False)
    missing_pct = (missing / df.shape[0] * 100).round(2)
    missing_df = pd.concat([missing, missing_pct], axis=1, keys=['count', '%']).query('count > 0')
    if not missing_df.empty:
        print("\nПропущенные значения:")
        print(missing_df)
    else:
        print("\nПропущенных значений нет")
        
    # Дубликаты
    dupes = df.duplicated().sum()
    print(f"\nДубликаты: {dupes} ({dupes/df.shape[0]*100:.2f}%)")
    
    # Примеры
    print(f"\nПервые {sample_size} записей:")
    display(df.head(sample_size))


def analyze_column(df: pd.DataFrame, col: str, max_display: int = 50) -> None:
    """Полный анализ колонки с выводом всех уникальных значений"""
    print(f"\n{'-'*60}")
    print(f"Полный анализ колонки: {col}")
    
    # Проверка существования колонки
    if col not in df.columns:
        print(f"Колонка {col} не найдена!")
        return
    
    # Пропуски
    na_count = df[col].isna().sum()
    print(f"Пропуски: {na_count} ({na_count/len(df)*100:.1f}%)")

    # Количество уникальных значений
    unique_count = df[col].nunique(dropna=False)
    print(f"Уникальных значений: {unique_count}")
    
    # Вывод всех значений для категориальных данных
    if unique_count <= max_display:
        print("\nВсе значения:")
        print(df[col].unique())
    else:
        print(f"\nСлишком много значений (> {max_display}). Примеры:")
        print(df[col].dropna().sample(10).unique())
        
    # Частотный анализ для числовых колонок
    if pd.api.types.is_numeric_dtype(df[col]):
        print("\nОписательная статистика:")
        print(df[col].describe())
    else:
        print("\nТоп-10 значений:")
        print(df[col].value_counts(dropna=False).head(20))


generate_summary(sessions_raw, "Сырые данные сессий")
for col in sessions_raw.columns:
    analyze_column(sessions_raw, col, max_display=100)

generate_summary(hits_raw, "Сырые данные событий")
for col in hits_raw.columns:
    if col == "event_action":
        analyze_column(hits_raw, col, max_display=300)
    else:
        analyze_column(hits_raw, col, max_display=100)
        


================================================== СЫРЫЕ ДАННЫЕ СЕССИЙ ==================================================
Общее количество записей: 1,860,042
Количество признаков: 18

Типы данных:
        count
object     17
int64       1

Пропущенные значения:
                 count     %
device_model   1843704 99.12
utm_keyword    1082061 58.17
device_os      1070138 57.53
utm_adcontent   335615 18.04
utm_campaign    219603 11.81
device_brand    118678  6.38
utm_source          97  0.01

Дубликаты: 0 (0.00%)

Первые 3 записей:


,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.163775...,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867...,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.164064...,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk



------------------------------------------------------------
Полный анализ колонки: session_id
Пропуски: 0 (0.0%)
Уникальных значений: 1860042

Слишком много значений (> 100). Примеры:
['2534717050643830177.1640000264.1640000264'
 '6667187475921648639.1631468547.1631468547'
 '2504003374430965652.1633046444.1633046444'
 '6792760087583689880.1625328794.1625328794'
 '427192146278500396.1640253483.1640253483'
 '1100191552804305545.1640577710.1640577710'
 '8539885537308799979.1640592363.1640592363'
 '8924143543872310578.1629146419.1629146419'
 '6711593787048718140.1640099644.1640099644'
 '2175326075774369474.1630423881.1630423881']

Топ-10 значений:
session_id
9055434745589932991.1637753792.1637753792    1
6294640354816492625.1638728787.1638728809    1
6294832296889291185.1623245519.1623245519    1
6294832296889291185.1623161065.1623161065    1
6294832296889291185.1623156608.1623156608    1
6294832296889291185.1623069105.1623069105    1
6294825034110946181.1634421637.1634421637    1
629481

,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,5639623078712724064.164025...,2021-12-23,597864.00,30,event,NaN,sberauto.com/cars?utm_sour...,quiz,quiz_show,NaN,None
1,7750352294969115059.164027...,2021-12-23,597331.00,41,event,NaN,sberauto.com/cars/fiat?cit...,quiz,quiz_show,NaN,None
2,885342191847998240.1640235...,2021-12-23,796252.00,49,event,NaN,sberauto.com/cars/all/volk...,quiz,quiz_show,NaN,None



------------------------------------------------------------
Полный анализ колонки: session_id
Пропуски: 0 (0.0%)
Уникальных значений: 1734610

Слишком много значений (> 100). Примеры:
['1122782767248919937.1640844675.1640844675'
 '8504924898641414249.1625655401.1625655405'
 '9151749872534384362.1640872243.1640872243'
 '1719912895044858538.1640410910.1640410910'
 '4835419155796497002.1633587818.1633587818'
 '3273100011322065205.1630388535.1630388535'
 '41754604237031782.1632176497.1632176497'
 '7629290803402572035.1626800386.1626800386'
 '6434935169467263360.1623435611.1623435611'
 '8173549321151398445.1635429995.1635429995']

Топ-10 значений:
session_id
5442565791571325612.1632449195.1632449195    768
6568868914238486437.1632270313.1632270313    678
5959671972744778783.1632490527.1632490600    548
7452598043578978502.1632358598.1632358598    514
3070792010704358528.1629752408.1629752408    498
8115026869866033734.1629319807.1629319807    496
686125592720823356.1634587719.1634587719  

## Обработка событий (Hits)

In [67]:
def clean_hits(hits: pd.DataFrame) -> pd.DataFrame:
    # 0) Копируем, чтобы не менять оригинал
    hits = hits_raw.copy()
    
    # 1) Удаляем полностью пустые/однотипные колонки
    hits = hits.drop(columns=['event_value', 'hit_type'])
    
    # 2) Обработка пропусков
    hits['hit_referer'] = hits['hit_referer'].fillna('unknown')
    hits['event_label'] = hits['event_label'].fillna('unknown')
    
    # 3) Преобразуем hit_time в секунды и оставляем NaN
    hits['hit_time_s'] = pd.to_numeric(hits['hit_time'], errors='coerce')
    
    # 4) Фильтруем невалидные времена
    hits = hits[hits['hit_time_s'].between(0, 86400)]
    
    return hits


def process_hits(hits: pd.DataFrame, target_actions: list) -> pd.DataFrame:

    # Агрегация по сессии
    hits_agg = (
        hits
        .groupby('session_id')
        .agg(
            # длительность в секундах
            session_duration_sec=pd.NamedAgg(
                column='hit_time_s',
                aggfunc=lambda x: (x.max(skipna=True) - x.min(skipna=True)) if x.notna().any() else 0
            ),
            # общее число хитов
            num_hits=pd.NamedAgg(
                column='hit_number',
                aggfunc='count'
            ),
            # число целевых событий
            num_target_events=pd.NamedAgg(
                column='event_action',
                aggfunc=lambda s: s.isin(target_actions).sum()
            ),
            # число уникальных страниц, которые посетил пользователь
            num_unique_pages=pd.NamedAgg(
                column='hit_page_path',
                aggfunc=lambda x: x.nunique()
            ),
            # максимальный порядковый номер события (глубина сессии)
            max_hit_number=pd.NamedAgg(
                column='hit_number',
                aggfunc='max'
            ),
            # среднее и медианное время между хитами
            avg_time_between=pd.NamedAgg(
                column='hit_time_s',
                aggfunc=lambda x: x.sort_values().diff().mean(skipna=True) if x.notna().sum()>1 else 0
            )

        )
        .reset_index()
    )

    return hits_agg


# Применение:
target_actions = [
    # Заявки и формы
    'sub_submit_success',        # отправка основной формы заявки
    'sub_car_request_submit_click',  # запрос автомобиля
    'form_request_call_sent',    # отправка формы обратного звонка
    'survey_buy_after_sell_start',    # начало опроса (важно считать как конверсию?)
    'request_success',           # успешный запрос (кредит/подписка)
    'request',                   # общий запрос — можно включить, если он всегда финальный
    
    # Подтверждения и создание объявлений
    'success_ad_creation',       # успешное создание объявления
    'add_ad_publish_success',    # публикация объявления
    
    # Звонки и контактные данные
    'click_on_request_call',     # клик «заказать звонок»
    'phone_entered_on_form_request_call',  # ввод телефона в форме звонка
    'name_entered_on_form_request_call',   # ввод имени в форме звонка
    
    # Чат и консьерж-сервис
    'chat established',          # чат установлен
    'client initiate chat',      # пользователь начал чат
    'auto_subscription_click',   # интерес к автоподписке (финальный клик)
    'click_on_subscription',     # клик «оформить подписку»
]

hits_processed = (
    sessions_raw
    .pipe(clean_hits)
    .pipe(process_hits, target_actions=target_actions)
)

# Проверка результата
print("\nПроверка hits_processed:")
print("\nКолонки итогового датафрейма:", hits_processed.columns.tolist())
display(hits_processed.sample(5))


Проверка hits_processed:

Колонки итогового датафрейма: ['session_id', 'session_duration_sec', 'num_hits', 'num_target_events', 'num_unique_pages', 'max_hit_number', 'avg_time_between']


,session_id,session_duration_sec,num_hits,num_target_events,num_unique_pages,max_hit_number,avg_time_between
519247,6910238193445223126.163812...,21638.00,2,0,1,4,21638.00
140104,2593198557997113349.163532...,70633.00,10,0,2,15,7848.11
530630,7043427173700375741.163987...,82830.00,4,0,2,8,27610.00
643596,8331340118894828027.163891...,0.00,1,0,1,3,0.00
62504,1713607797155400141.164059...,10198.00,3,0,1,9,5099.00


## Обработка сессий (Sessions)

In [69]:
import pandas as pd
import numpy as np

# Константы для заполнения пропусков
FILL_VALUES = {
    'utm_source':    'unknown',
    'utm_medium':    'unknown',
    'utm_campaign':  'unknown',
    'utm_adcontent': 'unknown',
    'utm_keyword':   'unknown',
    'device_os':     'unknown',
    'device_brand':  'unknown',
    'device_browser':'unknown'
}

def process_sessions(df: pd.DataFrame, top_n_cities: int = 20, rare_thresh: float = 0.01) -> pd.DataFrame:
    df = df.copy()
    
    # 1) Заполнение пропусков единообразно
    for col, fill_val in FILL_VALUES.items():
        df[col] = df[col].fillna(fill_val)
    
    # 2) Объединим редкие utm_campaign и utm_adcontent в категорию 'other'
    for col, other_label in [('utm_campaign', 'other_campaign'), ('utm_adcontent', 'other_adcontent')]:
        freqs = df[col].value_counts(normalize=True)
        rare = freqs[freqs < rare_thresh].index
        df[col] = df[col].where(~df[col].isin(rare), other_label)
    
    # 3) Парсинг даты+времени и базовые фичи
    df['session_start'] = pd.to_datetime(
        df['visit_date'].astype(str) + ' ' + df['visit_time'].astype(str),
        errors='coerce'
    )
    # Отбрасываем строки с некорректными датами
    df = df[df['session_start'].notna()]
    
    df['visit_weekday'] = df['session_start'].dt.weekday
    df['visit_hour']    = df['session_start'].dt.hour
    
    # 4) Фаза дня и выходной день
    df['timeofday'] = pd.cut(
        df['visit_hour'],
        bins=[-1, 5, 11, 17, 23],
        labels=['night','morning','afternoon','evening']
    )
    df['is_weekend'] = df['visit_weekday'].isin([5, 6]).astype(int)
    
    # 5) Тип трафика
    paid    = ['cpc','cpm','cpa']
    organic = ['organic','referral','(none)']
    df['traffic_type'] = np.where(
        df['utm_medium'].isin(paid), 'paid',
        np.where(df['utm_medium'].isin(organic), 'organic', 'other')
    )
    
    # 6) Группировка городов: топ-N + остальное
    top_cities = df['geo_city'].value_counts().nlargest(top_n_cities).index
    df['geo_city_group'] = np.where(
        df['geo_city'].isin(top_cities),
        df['geo_city'],
        'other'
    )
    
    # 7) Флаги и трансформации
    df['is_first_visit']   = (df['visit_number'] == 1).astype(int)
    df['log_visit_number'] = np.log1p(df['visit_number'])
    df['is_mobile']        = (df['device_category'] == 'mobile').astype(int)
    
    # 8) Приведение категорий к 'category'
    cat_cols = [
        'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent',
        'utm_keyword', 'device_os', 'device_brand', 'device_browser',
        'device_category', 'geo_country', 'geo_city_group',
        'traffic_type', 'timeofday'
    ]
    for c in cat_cols:
        df[c] = df[c].astype('category')
    
    # 9) Удаляем промежуточные колонки
    return df.drop(columns=['visit_date', 'visit_time'])


def handle_device_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    # 1) Извлечение width/height из device_screen_resolution
    #    Некорректные и '(not set)' превращаем в NaN
    resolution = (
        df['device_screen_resolution']
        .replace({'(not set)': np.nan, 'unknown': np.nan})
        .str.extract(r'(?P<screen_width>\d+)x(?P<screen_height>\d+)')
    )
    df['screen_width']  = resolution['screen_width'].astype(float)
    df['screen_height'] = resolution['screen_height'].astype(float)
    
    # 2) Приводим к числу, оставляем NaN для редких/ошибочных
    #    При необходимости заполнять можно медианой:
    # df['screen_width'].fillna(df['screen_width'].median(), inplace=True)
    # df['screen_height'].fillna(df['screen_height'].median(), inplace=True)
    
    # 3) Удаляем более не нужные колонки
    return df.drop(columns=['device_screen_resolution', 'device_model'])


# Пример объединённого пайплайна
sessions_processed = (
    sessions_raw
    .pipe(process_sessions, top_n_cities=20, rare_thresh=0.01)
    .pipe(handle_device_data)
)

# Проверка результата
print("\nПроверка sessions_processed:")
print("\nКолонки итогового датафрейма:", sessions_processed.columns.tolist())
display(sessions_processed.sample(5))



Проверка sessions_processed:

Колонки итогового датафрейма: ['session_id', 'client_id', 'visit_number', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'utm_keyword', 'device_category', 'device_os', 'device_brand', 'device_browser', 'geo_country', 'geo_city', 'session_start', 'visit_weekday', 'visit_hour', 'timeofday', 'is_weekend', 'traffic_type', 'geo_city_group', 'is_first_visit', 'log_visit_number', 'is_mobile', 'screen_width', 'screen_height']


,session_id,client_id,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_browser,geo_country,geo_city,session_start,visit_weekday,visit_hour,timeofday,is_weekend,traffic_type,geo_city_group,is_first_visit,log_visit_number,is_mobile,screen_width,screen_height
1546472,765514884901939978.1624943...,178235323.1624943370,1,hTjLvqNxGggkGnxSCaTm,stories,other_campaign,JNHcPlZPxEMWDnRiyoBf,unknown,mobile,unknown,Apple,Safari (in-app),Russia,Moscow,2021-06-29 08:00:00,1,8,morning,0,other,Moscow,1,0.69,1,375.00,667.00
782780,4254547892975153761.162186...,990589124.1621865057,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,unknown,mobile,unknown,Asus,Chrome,Russia,Moscow,2021-05-24 17:00:00,0,17,afternoon,0,organic,Moscow,1,0.69,1,360.00,720.00
1276814,6455930494332373721.163230...,1503138452.1632307929,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,Safari (in-app),Russia,Grozny,2021-09-22 13:52:10,2,13,afternoon,0,organic,Grozny,1,0.69,1,414.00,896.00
1464450,7293202731962696666.163653...,1698081086.1636533210,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,Chrome,Russia,Novosibirsk,2021-11-10 11:33:31,2,11,morning,0,other,Novosibirsk,1,0.69,1,360.00,640.00
1547363,7659152323742463164.163325...,1783285365.1632040124,2,kjsLglQLzykiRbcDiGcD,cpc,other_campaign,unknown,unknown,mobile,unknown,Samsung,Chrome,Russia,Saint Petersburg,2021-10-03 11:00:00,6,11,morning,1,paid,Saint Petersburg,0,1.10,1,412.00,869.00


## Объединение данных

In [71]:
def merge_datasets(sessions: pd.DataFrame, hits: pd.DataFrame) -> pd.DataFrame:
    
    # 1) Левый merge — сохраняем все сессии
    merged = sessions.merge(
        hits,
        on="session_id",
        how="left",
        validate="1:1"   # проверка, что по одной сессии не прилетает дубликат
    )

    # 2) Заполняем нулями и приводим к целочисленному типу
    for col in ["session_duration_sec", "num_hits", "num_target_events",
                "num_unique_pages", "max_hit_number", "avg_time_between"]:
        if col in merged:
            if col == "avg_time_between":
                # заменяем NaN на 0 для однохитовых сессий
                merged[col] = merged[col].fillna(0.0)
            else:
                merged[col] = merged[col].fillna(0)
            # приведение типов
            if col != "avg_time_between":
                merged[col] = merged[col].astype(int)

    # 3) Лог‑преобразования новых счётчиков
    merged["log_session_duration_sec"] = np.log1p(merged["session_duration_sec"])
    merged["log_num_hits"] = np.log1p(merged["num_hits"])
    merged["log_num_target_events"] = np.log1p(merged["num_target_events"])
    merged["log_num_unique_pages"] = np.log1p(merged["num_unique_pages"])
    merged["log_max_hit_number"] = np.log1p(merged["max_hit_number"])
    merged["log_avg_time_between"] = np.log1p(merged["avg_time_between"])

    # 4) Итоговые признаки готовы
    return merged


# Пайплайн объединения
data_processed = merge_datasets(sessions_processed, hits_processed)

# Проверка результата
print("\nПроверка data_processed:")
print("\nКолонки итогового датафрейма:", data_processed.columns.tolist())
display(data_processed.sample(5))


Проверка data_processed:

Колонки итогового датафрейма: ['session_id', 'client_id', 'visit_number', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'utm_keyword', 'device_category', 'device_os', 'device_brand', 'device_browser', 'geo_country', 'geo_city', 'session_start', 'visit_weekday', 'visit_hour', 'timeofday', 'is_weekend', 'traffic_type', 'geo_city_group', 'is_first_visit', 'log_visit_number', 'is_mobile', 'screen_width', 'screen_height', 'session_duration_sec', 'num_hits', 'num_target_events', 'num_unique_pages', 'max_hit_number', 'avg_time_between', 'log_session_duration_sec', 'log_num_hits', 'log_num_target_events', 'log_num_unique_pages', 'log_max_hit_number', 'log_avg_time_between']


,session_id,client_id,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_browser,geo_country,geo_city,session_start,visit_weekday,visit_hour,timeofday,is_weekend,traffic_type,geo_city_group,is_first_visit,log_visit_number,is_mobile,screen_width,screen_height,session_duration_sec,num_hits,num_target_events,num_unique_pages,max_hit_number,avg_time_between,log_session_duration_sec,log_num_hits,log_num_target_events,log_num_unique_pages,log_max_hit_number,log_avg_time_between
839939,4508806078544372202.162495...,1049788221.1623351786,6,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,unknown,mobile,unknown,Samsung,Chrome,Russia,Moscow,2021-06-29 12:00:00,1,12,afternoon,0,organic,Moscow,0,1.95,1,360.00,800.00,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
696379,3867400838010826274.163885...,900449426.1638854178,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,pHqgLcdBQPSLcrTpLkdr,desktop,Linux,unknown,Chrome,Russia,Mozhaysk,2021-12-07 08:16:18,1,8,morning,0,paid,other,1,0.69,0,1280.00,720.00,16528,3,0,1,7,8264.00,9.71,1.39,0.00,0.69,2.08,9.02
1767330,8642341617080372528.163830...,2012201961.1638305072,2,kjsLglQLzykiRbcDiGcD,cpc,unknown,unknown,hAmNSZmQkKQKAjZEGlgb,mobile,iOS,Apple,Safari,Russia,(not set),2021-11-30 23:44:36,1,23,evening,0,paid,(not set),0,1.10,1,375.00,812.00,33287,14,0,2,20,2560.54,10.41,2.71,0.00,1.10,3.04,7.85
1443948,7202528722435131955.163232...,1676969398.1632324147,1,RmEBuqrriAfAVsLQQmhk,cpc,other_campaign,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,tablet,Android,Sony,Chrome,Russia,Moscow,2021-09-22 18:23:32,2,18,evening,0,paid,Moscow,1,0.69,0,1280.00,800.00,4494,6,0,1,8,898.80,8.41,1.95,0.00,0.69,2.20,6.80
190762,1608257905840549865.162523...,374451723.1624698857,3,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,unknown,mobile,unknown,Apple,Safari,Russia,Samara,2021-07-02 17:00:00,4,17,afternoon,0,other,Samara,0,1.39,1,414.00,896.00,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Экспорт данных

In [76]:
print("\nСохраняем обработанные данные...")
data_out.mkdir(parents=True, exist_ok=True)
data_processed.to_pickle(data_out / 'data_processed.pkl')
print(f"Данные сохранены в: {data_out}")


Сохраняем обработанные данные...
Данные сохранены в: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/processed_data
